In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
import numpy as np

def calculate_median_of_each_burst(sample):
    bursts = []
    current_burst = []
    for packet in sample:
        if not current_burst or (packet > 0) == (current_burst[-1] > 0):
            current_burst.append(packet)
        else:
            bursts.append(current_burst)
            current_burst = [packet]
    if current_burst:
        bursts.append(current_burst)

    medians = []
    for burst in bursts:
        timestamps = [abs(p) for p in burst]
        median_timestamp = np.median(timestamps) if timestamps else 0
        medians.append(median_timestamp)

    return medians

In [3]:
import numpy as np
import pickle

USE_SUBLABEL = False
URL_PER_SITE = 10
TOTAL_URLS = 950

path = '/gdrive/MyDrive/mon_standard.pkl'

# Load the pickle file
print("Loading datafile...")
with open(path, 'rb') as fi:
    data = pickle.load(fi)

# 전체 데이터셋에 대한 MED 계산
X_MED = []
y = []

for i in range(TOTAL_URLS):
    label = i if USE_SUBLABEL else i // URL_PER_SITE
    for sample in data[i]:
        medians = calculate_median_of_each_burst(sample)
        X_MED.append(medians)
        y.append(label)

print(f'Total samples processed: {len(X_MED)}')

Loading datafile...
Total samples processed: 19000


In [ ]:
print(X_MED[0])

[0.07, 0.14, 0.31, 0.31, 0.51, 0.51, 0.815, 0.89, 1.13, 1.13, 1.3, 1.3, 1.47, 1.47, 1.72, 1.845, 2.21, 2.21, 2.38, 2.48, 2.68, 2.68, 2.98, 2.98, 3.15, 3.15, 3.17, 3.17, 3.17, 3.48, 3.68, 3.69, 3.69, 3.71, 3.73, 3.78, 3.78, 3.78, 3.81, 3.81, 3.82, 3.82, 3.82, 3.82, 3.87, 3.87, 3.87, 3.87, 3.88, 3.88, 3.9, 3.9, 3.91, 3.92, 3.93, 3.93, 3.95, 3.97, 3.97, 3.97, 3.99, 4.0, 4.01, 4.03, 4.04, 4.04, 4.05, 4.05, 4.05, 4.06, 4.09, 4.09, 4.1, 4.1, 4.1, 4.1, 4.12, 4.13, 4.13, 4.14, 4.16, 4.17, 4.175, 4.21, 4.21, 4.34, 4.38, 4.38, 4.57, 4.57, 4.64, 4.64, 4.64, 4.65, 4.65, 4.65, 4.69, 4.69, 4.69, 4.69, 4.69, 4.73, 4.76, 4.77, 4.84, 4.84, 4.85, 4.86, 5.04, 5.04, 5.05, 5.06, 5.08, 5.08, 5.1, 5.12, 5.13, 5.22, 5.23, 5.23, 5.46, 5.545, 5.55, 5.55, 5.72, 5.82, 6.0, 6.0, 6.01, 6.03, 6.03, 6.03, 6.04, 6.195, 6.46, 6.46, 6.720000000000001, 6.79, 7.0, 7.0, 7.1, 7.105, 7.41, 7.41, 7.63, 7.99, 8.21, 8.22, 8.39, 8.39, 8.62, 8.625, 8.92, 8.925, 9.22, 9.68]


In [ ]:
import numpy as np

# 데이터의 최대 길이 계산
max_length = max(len(seq) for seq in X_MED)

# 패딩만 수행하는 함수 정의
def pad_sequence(sequence, max_length):
    return sequence + [0] * (max_length - len(sequence))

# 각 시퀀스에 패딩만 적용
X_padded = [pad_sequence(seq, max_length) for seq in X_MED]

In [ ]:
from sklearn.model_selection import train_test_split

# 훈련 데이터와 테스트 데이터를 8:2 비율로 분할
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# SVM 모델 생성
svm_model = SVC(kernel='linear', random_state=42)  # 'linear' 커널 사용, 다른 커널도 선택 가능

# 훈련 데이터로 모델 훈련
svm_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.3468421052631579


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# SVM 모델 설정
svm_model = SVC(random_state=42)

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [10, 100, 150],  # 규제 매개변수
    'kernel': ['linear'],  # 커널 타입
    'gamma': ['scale'],  # 커널 계수
    # 'degree': [2, 3, 4] 등과 같이 'poly' 커널에 대한 추가 매개변수를 추가할 수 있습니다.
}

# GridSearchCV 설정
grid_search = GridSearchCV(svm_model, param_grid, cv=2, scoring='accuracy', verbose=2)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적의 매개변수와 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# SVM 모델 생성
svm_model = SVC(kernel='rbf', random_state=42)  # 'linear' 커널 사용, 다른 커널도 선택 가능

# 훈련 데이터로 모델 훈련
svm_model.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = svm_model.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.15447368421052632


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# SVM 모델 설정
svm_model = SVC(random_state=42)

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [10, 100, 150],  # 규제 매개변수
    'kernel': ['rbf'],  # 커널 타입
    'gamma': ['scale'],  # 커널 계수
    # 'degree': [2, 3, 4] 등과 같이 'poly' 커널에 대한 추가 매개변수를 추가할 수 있습니다.
}

# GridSearchCV 설정
grid_search = GridSearchCV(svm_model, param_grid, cv=3, scoring='accuracy', verbose=2)

# 그리드 서치 실행
grid_search.fit(X_train, y_train)

# 최적의 매개변수와 성능 출력
print("Best Parameters:", grid_search.best_params_)
print("Best Score:", grid_search.best_score_)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN 분류기 생성
knn = KNeighborsClassifier(n_neighbors=5)  # n_neighbors는 KNN에서 고려할 이웃의 수

# 훈련 데이터로 모델 훈련
knn.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = knn.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.205


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# RandomForest 분류기 생성
rf = RandomForestClassifier(n_estimators=100, random_state=42)  # n_estimators는 트리의 수

# 훈련 데이터로 모델 훈련
rf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측 수행
y_pred = rf.predict(X_test)

# 정확도 계산
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")

Test Accuracy: 0.3207894736842105
